# Clustering Crypto

In [2]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
# import hvplot.pandas
# import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from urllib.request import Request, urlopen
import json



Bad key "text.kerning_factor" on line 4 in
C:\Users\nites\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
!pip install -U altair

# Amazon SageMaker and related imports
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
import boto3  # AWS Python sdk

%matplotlib inline

Requirement already up-to-date: altair in c:\users\nites\anaconda3\envs\pyvizenv\lib\site-packages (4.1.0)


ModuleNotFoundError: No module named 'sagemaker'

### Fetching Cryptocurrency Data

In [4]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"


In [5]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

data = urlopen(Request(url)).read()
json_data = json.loads(data)
df_full = pd.DataFrame(json_data["Data"])
df_full.head()

,42,300,365,404,433,611,808,888,1337,2015,...,PPC,BCN,BURST,SC,POA,AE,UTOPIA,XLM,UMI,TNCC
Algorithm,Scrypt,N/A,X11,Scrypt,N/A,SHA-256,SHA-256,N/A,X13,X11,...,N/A,CryptoNight,Shabal256,Blake2b,Proof-of-Authority,N/A,N/A,N/A,N/A,N/A
BlockNumber,195131,0,NaN,47335,10821438,NaN,0,0,2754128,NaN,...,518416,0,385192,277516,0,314042,0,0,NaN,0
BlockReward,0,0,NaN,11.7691,2,NaN,0,0,0,NaN,...,1.64623,0,1660,30000,0,145.233,0,0,NaN,0
BlockTime,0,0,NaN,60,0,NaN,0,88,60,NaN,...,0,0,0,600,5,180,0,0,NaN,0
BuiltOn,N/A,7605,N/A,N/A,7605,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,7605,N/A,N/A,7605


In [6]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
df_transpose = df_full.T
df_transpose.head()

,Algorithm,BlockNumber,BlockReward,BlockTime,BuiltOn,CoinName,ContentCreatedOn,DecimalPlaces,FullName,FullyPremined,...,Rating,SmartContractAddress,SortOrder,Sponsored,Symbol,Taxonomy,TotalCoinSupply,TotalCoinsFreeFloat,TotalCoinsMined,Url
42,Scrypt,195131,0,0,N/A,42 Coin,1427211129,0,42 Coin (42),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,34,False,42,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",42,N/A,42,/coins/42/overview
300,N/A,0,0,0,7605,300 token,1517935016,18,300 token (300),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0xaec98a708810414878c3bcdf46aad31ded4a4557,2212,False,300,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",300,N/A,300,/coins/300/overview
365,X11,NaN,NaN,NaN,N/A,365Coin,1480032918,0,365Coin (365),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,916,False,365,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",2300000000,N/A,NaN,/coins/365/overview
404,Scrypt,47335,11.7691,60,N/A,404Coin,1466100361,0,404Coin (404),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,602,False,404,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",532000000,N/A,1.27383e+06,/coins/404/overview
433,N/A,10821438,2,0,7605,433 Token,1541597321,18,433 Token (433),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0x738505a5f31bf72e0b70298bca81150eb1b7c751,3505,False,433,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",1000000000,N/A,1.12518e+08,/coins/433/overview


### Data Preprocessing

In [7]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

df = df_transpose[["CoinName", "Algorithm", "IsTrading", "ProofType", "TotalCoinsMined", "TotalCoinSupply"]]
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.27383e+06,532000000
433,433 Token,N/A,False,N/A,1.12518e+08,1000000000


In [8]:
# Keep only cryptocurrencies that are trading

df = df.loc[df.IsTrading != False]
df.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.27383e+06,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000


In [9]:
# Keep only cryptocurrencies with a working algorithm
df = df.loc[df.Algorithm != "N/A"]
df.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.27383e+06,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0,0


In [10]:
# Remove the 'IsTrading' column
df = df.drop(columns = "IsTrading")
df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.27383e+06,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0,0


In [11]:
# Remove rows with at least 1 null value
# to check the null values -->  df.isnull().sum()
df = df.dropna()
df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [12]:
# Remove rows with cryptocurrencies having no coins mined
df = df.loc[df.TotalCoinsMined != 0]
df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
404,404Coin,Scrypt,PoW/PoS,1.27383e+06,532000000
1337,EliteCoin,X13,PoW/PoS,2.94819e+10,314159265359
BTCD,BitcoinDark,SHA-256,PoW/PoS,1288862,22000000
XPY,PayCoin,SHA-256,PoS,1.19953e+07,12500000


In [13]:
# Check Columns where there are 'N/A' text values
for i in df.columns:
    print (df[df[i] == "N/A"])


    


Empty DataFrame
Columns: [CoinName, Algorithm, ProofType, TotalCoinsMined, TotalCoinSupply]
Index: []
Empty DataFrame
Columns: [CoinName, Algorithm, ProofType, TotalCoinsMined, TotalCoinSupply]
Index: []
                          CoinName      Algorithm ProofType TotalCoinsMined  \
XPD                    PetroDollar       SHA-256D       N/A        63993275   
CREVA                   Creva Coin            X11       N/A        36390750   
UNC                         UnCoin            X11       N/A         8388608   
BCY                    BitCrystals   Counterparty       N/A       100000000   
SCOT                      Scotcoin   Counterparty       N/A      1000000000   
PASC                   Pascal Coin         Pascal       N/A        32543700   
NETKO                        Netko          Blake       N/A        10853165   
WGR                         Wagerr            PoS       N/A       200000000   
GRWI         Growers International           DPoS       N/A     1.22049e+06   
AURS  

In [14]:
# Dropping rows where there are "N/A" text values in ProofType
df = df.loc[df["ProofType"].str.contains("N/A") == False]

# Dropping rows where there are "N/A" text values in TotalCoinsSupply
df = df.loc[df["TotalCoinSupply"].str.contains("N/A") == False]


In [15]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 

crypto_name = df.CoinName
crypto_name.head()


42          42 Coin
404         404Coin
1337      EliteCoin
BTCD    BitcoinDark
XPY         PayCoin
Name: CoinName, dtype: object

In [16]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
cluster_df = df.drop(columns = "CoinName")
print(cluster_df.shape)
cluster_df.head()


(541, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,42,42
404,Scrypt,PoW/PoS,1.27383e+06,532000000
1337,X13,PoW/PoS,2.94819e+10,314159265359
BTCD,SHA-256,PoW/PoS,1288862,22000000
XPY,SHA-256,PoS,1.19953e+07,12500000


In [32]:
# Create dummy variables for text features
cluster_df_dummy = pd.get_dummies(cluster_df[["Algorithm", "ProofType"]])
# cluster_df_dummy = pd.get_dummies(cluster_df["ProofType"])
cluster_df_dummy.head()


,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_C31,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTCD,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XPY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Standardize data

cluster_df_scaled = StandardScaler().fit_transform(cluster_df_dummy)
cluster_df_scaled[0:5]

array([[-0.04303315, -0.04303315, -0.04303315, -0.14406497, -0.04303315,
        -0.04303315, -0.04303315, -0.04303315, -0.06091449, -0.06091449,
        -0.45857478, -0.04303315, -0.07467401, -0.04303315, -0.04303315,
        -0.86649239, -0.04303315,  1.44009154, -0.04303315, -0.04303315,
        -0.04303315, -0.04303315, -0.06091449, -0.04303315, -0.04303315,
        -0.04303315, -0.04303315],
       [-0.04303315, -0.04303315, -0.04303315, -0.14406497, -0.04303315,
        -0.04303315, -0.04303315, -0.04303315, -0.06091449, -0.06091449,
        -0.45857478, -0.04303315, -0.07467401, -0.04303315, -0.04303315,
        -0.86649239, -0.04303315,  1.44009154, -0.04303315, -0.04303315,
        -0.04303315, -0.04303315, -0.06091449, -0.04303315, -0.04303315,
        -0.04303315, -0.04303315],
       [-0.04303315, -0.04303315, -0.04303315, -0.14406497, -0.04303315,
        -0.04303315, -0.04303315, -0.04303315, -0.06091449, -0.06091449,
        -0.45857478, -0.04303315, -0.07467401, -0.0430

### Reducing Dimensions Using PCA

In [19]:
# Use PCA to reduce dimensions to 3 principal components

# Initialize PCA model
pca = PCA(n_components=2)

cluster_pca = pca.fit_transform(cluster_df_scaled)

In [20]:
# Create a DataFrame with the principal components data
df_cluster_pca = pd.DataFrame(data = cluster_pca, 
                              columns=["PC 1", "PC 2"],                              index = cluster_df.index
                              )
                    
print(cluster_pca.shape)
df_cluster_pca.head()

(541, 2)


,PC 1,PC 2
42,1.500522,-0.848485
404,1.500522,-0.848485
1337,1.500522,-0.848485
BTCD,1.500522,-0.848485
XPY,0.463350,2.322584


In [21]:
# Fetch the explained variance
pca.explained_variance_ratio_.sum()

0.10688769956544175

### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [22]:
import altair as alt
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_cluster_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

alt.Chart(df_elbow, title = "Elbow Curve").mark_line().encode(
    x='k',
    y='inertia'
)


alt.Chart(...)

Running K-Means with `k= 3`

In [23]:
# Initialize the K-Means model
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(df_cluster_pca)

# Predict clusters
predictions = model.predict(df_cluster_pca)

# Add the predicted class columns
df_cluster_pca["class"] = model.labels_
df_cluster_pca.head()
# df_cluster_pca.shape

,PC 1,PC 2,class
42,1.500522,-0.848485,0
404,1.500522,-0.848485,0
1337,1.500522,-0.848485,0
BTCD,1.500522,-0.848485,0
XPY,0.463350,2.322584,2


In [24]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_clustered = pd.DataFrame({
    "Algorithm" : cluster_df.Algorithm,
    "ProofType": cluster_df.ProofType,
    "TotalCoinsMined": cluster_df.TotalCoinsMined,
    "TotalCoinSupply": cluster_df.TotalCoinSupply,
    "principal component 1": df_cluster_pca["PC 1"],
    "principal component 2": df_cluster_pca["PC 2"], 
    "CoinName": crypto_name,
    "Class": df_cluster_pca["class"]
    }, index = cluster_df.index
    )

df_clustered.head(20)




,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,CoinName,Class
42,Scrypt,PoW/PoS,42,42,1.500522,-0.848485,42 Coin,0
404,Scrypt,PoW/PoS,1.27383e+06,532000000,1.500522,-0.848485,404Coin,0
1337,X13,PoW/PoS,2.94819e+10,314159265359,1.500522,-0.848485,EliteCoin,0
BTCD,SHA-256,PoW/PoS,1288862,22000000,1.500522,-0.848485,BitcoinDark,0
XPY,SHA-256,PoS,1.19953e+07,12500000,0.463350,2.322584,PayCoin,2
PRC,Scrypt,PoW,6268245,21000000,-1.389375,-0.358185,ProsperCoin,1
KOBO,X15,PoW/PoS,2.55965e+07,350000000,1.500522,-0.848485,KoboCoin,0
ARG,Scrypt,PoW,1.38329e+07,64000000,-1.389375,-0.358185,Argentum,1
BLU,Scrypt,PoW/PoS,647272520,0,1.500522,-0.848485,BlueCoin,0
XMY,Multiple,PoW,1747693250,2000000000,-1.389375,-0.358185,MyriadCoin,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [25]:
# Create a 3D-Scatter with the PCA data and the clusters
# fig = px.scatter_3d(
#     df_cluster_pca,
#     x="principal component 1",
#     y="principal component 2",
#     z="principal component 3",
#     color="class",
#     symbol="class",
#     width=800,
# )
# fig.update_layout(legend=dict(x=0, y=1))
# fig.show()

alt.Chart(df_cluster_pca).mark_circle(size=60).encode(
    
    x='principal component 1:Q',
    y='principal component 2:Q',
    color='class:N',
    shape = "class:N",
    tooltip=['CoinName', 'principal component 1', 'principal component 2', 'Algorithm']
).interactive()


ValueError: CoinName encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

#### Table of Tradable Cryptocurrencies

In [26]:
# Table with tradable cryptos

# columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
display(df_clustered[['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']])

,CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class
42,42 Coin,Scrypt,PoW/PoS,42,42,0
404,404Coin,Scrypt,PoW/PoS,532000000,1.27383e+06,0
1337,EliteCoin,X13,PoW/PoS,314159265359,2.94819e+10,0
BTCD,BitcoinDark,SHA-256,PoW/PoS,22000000,1288862,0
XPY,PayCoin,SHA-256,PoS,12500000,1.19953e+07,2
PRC,ProsperCoin,Scrypt,PoW,21000000,6268245,1
KOBO,KoboCoin,X15,PoW/PoS,350000000,2.55965e+07,0
ARG,Argentum,Scrypt,PoW,64000000,1.38329e+07,1
BLU,BlueCoin,Scrypt,PoW/PoS,0,647272520,0
XMY,MyriadCoin,Multiple,PoW,2000000000,1747693250,1


In [27]:
# Print the total number of tradable cryptocurrencies
print("The number of tradable cryptocurrencies is") 
df_clustered["CoinName"].count()

The number of tradable cryptocurrencies is


541

#### Scatter Plot with Tradable Cryptocurrencies

In [28]:
# Scale data to create the scatter plot
df_clustered['TotalCoinsMined'] = df_clustered['TotalCoinsMined'].astype(float) / 100000000

df_clustered['TotalCoinSupply'] = df_clustered['TotalCoinSupply'].astype(float) / 100000000


In [29]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
# df_clustered.hvplot(
#     kind="scatter", 
#     x="TotalCoinsMined", 
#     y="TotalCoinSupply", 
#     c='Class',
#     rot= 90, 
#     colormap="inferno", 
#     hover_cols=['CoinName']
# )
alt.Chart(df_clustered).mark_circle(size=60).encode(
    
    x='TotalCoinsMined',
    y='TotalCoinSupply',
    color='class:N',
    tooltip=['class', 'principal component 1', 'principal component 2', 'principal component 3']
).interactive()

ValueError: class encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.